In [1]:
# imports
import torch
from torch import nn, optim
import torch.nn.functional as F

In [2]:
# load the dataset
from torchvision import datasets, transforms

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
# Download and load the training data
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download and load the test data
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

c:\Users\micha\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [5]:
# define the model class, Classifier
class Classifier(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        
        # fully-connected layers
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        
        # dropout module, used only during training, with probability = 0.2
        self.dropout = nn.Dropout(p=0.2)
        
    def forward(self, x: torch.tensor):
        
        # make sure the input tensor is flattened
        x = x.view(x.shape[0], -1)
        
        # define the forward pass, with dropout
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        
        # define the final layer, without dropout at the output
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x

In [6]:
# instantiate the model, loss function (criterion), and optimizer
model = Classifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

In [7]:
# train the model

epochs = 5

train_losses = []
test_losses = []

for e in range(epochs):
    
    tot_train_loss = 0
    
    for images, labels in trainloader:
        
        # forward pass and loss
        log_ps = model(images)
        loss = criterion(log_ps, labels)
        
        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # add training loss to tally
        tot_train_loss += loss.item()
        
    else:
        # perform validation (evaluate accuracy)
        test_loss = 0
        accuracy = 0
        
        with torch.no_grad():
            
            # SET MODEL TO EVALUATION MODE!
            model.eval()
            
            for images, labels in testloader:
            
                # forward pass and tally test loss
                log_ps = model(images)
                test_loss += criterion(log_ps, labels)
                
                # compute test accuracy
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
        
        # SET MODEL BACK TO TRAINING MODE!
        model.train()
        
        train_losses.append(tot_train_loss/len(trainloader))
        test_losses.append(test_loss/len(testloader))

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(train_losses[-1]),
              "Test Loss: {:.3f}.. ".format(test_losses[-1]),
              "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))
            
        

Epoch: 1/5..  Training Loss: 0.610..  Test Loss: 0.506..  Test Accuracy: 0.814
Epoch: 2/5..  Training Loss: 0.485..  Test Loss: 0.458..  Test Accuracy: 0.839
Epoch: 3/5..  Training Loss: 0.454..  Test Loss: 0.423..  Test Accuracy: 0.850
Epoch: 4/5..  Training Loss: 0.431..  Test Loss: 0.403..  Test Accuracy: 0.852
Epoch: 5/5..  Training Loss: 0.423..  Test Loss: 0.428..  Test Accuracy: 0.846
